In [1]:
import sys
sys.path.append('..')

In [2]:
import src
from shapely import wkt
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

# Polígonos del RENABAP
# https://datos.gob.ar/dataset/habitat-registro-nacional-barrios-populares
renabap = gpd.read_file('https://archivo.habitat.gob.ar/dataset/ssisu/renabap-datos-barrios-geojson')

In [3]:
mask = wkt.loads(
    "POLYGON((-58.1058 -34.824, -57.8183 -34.824, -57.8183 -35.0353, -58.1058 -35.0353, -58.1058 -34.824))"
)

In [21]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
import numpy as np

def loop_polys_get_panos(gdf, buffer_dist, dist_points, mask=None, max_workers=10, chunk_size=100):
    gdf_buffered = gdf.copy()

    # Apply optional mask
    if mask is not None:
        gdf_buffered = gdf_buffered[gdf_buffered.intersects(mask)]

    # Buffer in meters (EPSG:3857), then back to WGS84
    gdf_buffered['geometry'] = gdf_buffered.to_crs(3857).buffer(buffer_dist).to_crs(4326)

    # Union and explode to get disjoint polygons
    gdf_buffered = gpd.GeoDataFrame(
        [gdf_buffered.union_all()], columns=['geometry']
    ).reset_index(drop=True).explode(index_parts=False)

    # Step 1: Generate all grid points
    all_points = []
    for geom in tqdm(gdf_buffered.geometry, desc="Generating Grid Points"):
        points = src.utils.create_point_grid(geom, dist_points)
        all_points.append(points)

    points_gdf = pd.concat(all_points).reset_index(drop=True)

    # Step 2: Submit one job per point (GeoDataFrame with one row)
    def get_pano_for_point(point_geom):
        point_gdf = gpd.GeoDataFrame(geometry=[point_geom], crs=4326)
        return src.sv.get_panos(point_gdf)

    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(get_pano_for_point, geom) for geom in points_gdf.geometry]
        for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching Panoramas"):
            result = future.result()
            if result is not None and not result.empty:
                results.append(result)

    return pd.concat(results).reset_index(drop=True) if results else pd.DataFrame()

In [5]:
%%time
panos = loop_polys_get_panos(renabap, buffer_dist=500, dist_points=50, mask=mask)

Fetching Panoramas: 100%|████████████████████████████████████| 59469/59469 [19:14<00:00, 51.50it/s]


CPU times: user 9min 27s, sys: 2min 25s, total: 11min 52s
Wall time: 19min 19s


In [20]:
panos.head()

,pano_id,lat,lon,date
0,RkDOYU5dvUfjc1GoKdie2A,-35.042131,-58.092130,None
1,gW0r_VJT-DDgYDs0kbYkVg,-35.041657,-58.092271,None
2,ak29enRDqAz9ytfzy84Wew,-35.042191,-58.092266,None
3,QHtFhxIquzlsJwYrbKAouA,-35.041580,-58.092272,None
4,CceD6CZ4pQoGi38rPDc7gA,-35.041734,-58.092271,None


In [7]:
total = renabap.union_all().area
partial = renabap[renabap.intersects(mask)].union_all().area

In [8]:
partial/total

0.03237114506298897